In [1]:
import torch.nn as nn

from datasets import *
from evaluation import *
from models import *
from pruning.unstructured import *
from pruning.structured import *

c:\Users\prabh\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
c:\Users\prabh\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Importing C:\Users\prabh\Smart-pruner-dev\pruning\unstructured\L1norm.py
Importing C:\Users\prabh\Smart-pruner-dev\pruning\Train.py
Importing C:\Users\prabh\Smart-pruner-dev\pruning\structured\decay_pruning.py


In [2]:
lenet = Lenet()
mnist = MNIST()
train_loader, test_loader = mnist.get_dataloader()

In [3]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lenet.parameters(), lr=0.01)

In [4]:
#for only training of any model (pruned or unpruned)
trainer=Trainer(lenet, 5, train_loader, criterion, optimizer)
trainer.train()

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 3.5230 | 
Epoch: 2 | train_loss: 2.3864 | 
Epoch: 3 | train_loss: 1.8727 | 
Epoch: 4 | train_loss: 1.6373 | 
Epoch: 5 | train_loss: 1.5521 | 


Lenet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [6]:
lenet = Lenet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lenet.parameters(), lr=0.01)
trainer = UnstructuredL1normPrune(lenet, 5, train_loader, criterion, optimizer, 0.9)
#train and prune the model and return both the original and pruned model
model, pruned_model = trainer.train_prune_retrain()

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 3.4671 | 
Epoch: 2 | train_loss: 2.3508 | 
Epoch: 3 | train_loss: 1.8642 | 
Epoch: 4 | train_loss: 1.7090 | 
Epoch: 5 | train_loss: 1.5280 | 
Training is done
Pruning is done


In [7]:
print(accuracy(model, test_loader))
print(accuracy(pruned_model, test_loader))
print(non_zero_weights(model))
print(non_zero_weights(pruned_model))
print(compression_ratio(model, pruned_model))

[0.9876]
[0.3187]
44190
4419
0.1


In [13]:
lenet = Lenet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lenet.parameters(), lr=0.01)
trainer = DecayPrune(lenet, 5, train_loader, criterion, optimizer, 0.9,0.2,False) #init pruning rate = 0.5, decay = 0.1
#train and prune the model and return both the original and pruned model
decay_model, decay_pruned_model = trainer.train_prune_retrain()

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 3.5038 | 
Epoch: 2 | train_loss: 2.3397 | 
Epoch: 3 | train_loss: 1.8304 | 
Epoch: 4 | train_loss: 1.7377 | 
Epoch: 5 | train_loss: 1.5686 | 
Training is done
Pruning is done


In [14]:
print(accuracy(decay_model, test_loader))
print(accuracy(decay_pruned_model, test_loader))
print(non_zero_weights(decay_model))
print(non_zero_weights(decay_pruned_model))
print(compression_ratio(decay_model, decay_pruned_model))

[0.9806]
[0.491]
44190
14163
0.32050237610319077


In [15]:
#with reverse =True
lenet = Lenet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lenet.parameters(), lr=0.01)
trainer = DecayPrune(lenet, 5, train_loader, criterion, optimizer, 0.9,0.2,True) #init pruning rate = 0.5, decay = 0.1
#train and prune the model and return both the original and pruned model
decay_model, decay_pruned_model = trainer.train_prune_retrain()

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 3.4717 | 
Epoch: 2 | train_loss: 2.3084 | 
Epoch: 3 | train_loss: 1.9085 | 
Epoch: 4 | train_loss: 1.5701 | 
Epoch: 5 | train_loss: 1.2904 | 
Training is done
Pruning is done


In [16]:
print(accuracy(decay_model, test_loader))
print(accuracy(decay_pruned_model, test_loader))
print(non_zero_weights(decay_model))
print(non_zero_weights(decay_pruned_model))
print(compression_ratio(decay_model, decay_pruned_model))

[0.9837]
[0.7749]
44190
14163
0.32050237610319077
